In [ ]:
import streamlit as st
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

# ดึงข้อมูลจาก LakeFS (parquet file)
df2 = pd.read_parquet(
    path=lakefs_s3_path,
    storage_options=storage_options
)

# แสดงข้อมูลใน Streamlit
st.title("Weather Dashboard")

# แสดงข้อมูลในตาราง
st.write("Weather Metrics", df2)

# แสดง 3 metrics: Temperature, Humidity, Wind Speed
col1, col2, col3 = st.columns(3)
with col1:
    st.metric("Temperature (°C)", f"{df2['Temperature'].mean():.2f} °C")
with col2:
    st.metric("Humidity (%)", f"{df2['Humidity'].mean():.2f} %")
with col3:
    st.metric("Wind Speed (km/h)", f"{df2['Wind Speed'].mean():.2f} km/h")

# สร้างกราฟเพื่อแสดงแนวโน้มของอุณหภูมิ (Temperature Trend)
st.subheader("Temperature Trend")
time_series_data = {
    'Time': pd.date_range(start="2023-04-01", periods=10, freq='D'),
    'Temperature': df2['Temperature'].sample(10).values,  # ใช้ข้อมูลสุ่มจาก df2
}

temp_df = pd.DataFrame(time_series_data)

# แสดงกราฟแนวโน้มของอุณหภูมิ
st.line_chart(temp_df.set_index('Time'))

# รีเฟรชทุก 15 นาที
if st.button('Refresh Now'):
    st.experimental_rerun()
